Section 1: import xanes_regtools and other necessary packages

In [1]:
#%matplotlib inline
#import mpld3
#mpld3.enable_notebook()
%matplotlib ipympl
#%matplotlib notebook
#import scipy.ndimage as sn
from scipy.ndimage import gaussian_filter
import xanes_regtools_develop as xr
import xanes_image_utils as xiu
import matplotlib.pyplot as plt
import os
import h5py
import numpy as np
import skimage.morphology as skm
import scipy.ndimage as spn
import SimpleITK as sitk
from copy import deepcopy
from pathlib import Path
from importlib import reload
xr = reload(xr)
xiu = reload(xiu)
plt.rcParams['figure.figsize'] = [14, 8]

try:
    import imagej
    ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
    ijui = ij.ui()
    ijui.showUI()
except:
    print('Fiji is launched already!')
ij.py.run_macro("""run("Brightness/Contrast...");""")

#import imagej
#ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
#ijui = ij.ui()
#ijui.showUI()

from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')

def fiji_close_all():
    imp = WindowManager.getCurrentImage()
    while imp is not None:
        imp.close()
        imp = WindowManager.getCurrentImage()
fiji_close_all()        
print('\033[04m\033[01m\033[34m     Section 1 finished     ')   

     Section 1 finished     


Section 2: user inputs

In [2]:
#fn_template = '/media/Disk2/data/AIJUN_LI_Proposal_304474_June28_2019/multipos_2D_xanes_scan2_id_{0}_repeat_00_pos_{1}.h5'
fn_template = '/NSLS2/xf18id1/users/2020Q1/MATEUSZ_Proposal_305729/xanes_scan2_id_{}.h5'
scan_id = 53210   #21214, 21221
pos = 0

# if the data is a part of multipos scan
multipos = False
# these are [y_start, y_width, x_start, x_width]
roi = [470, 500, 700, 730]
# alignment section by section; sometime, you may see an abrupt misalignment between two chunks. You may try 
# different chunk_sz to correct such misalignment.
chunk_sz = 10
# if xanes images are already normalized
is_raw = False

# if use an alternative flat dataset
use_alternative_flat = False
# file name to the alternative flat dataset
flat_name = '/NSLS2/xf18id1/users/2019Q3/XU_ZHENGRUI_Proposal_305094/multipos_2D_xanes_scan2_id_26320_repeat_00_pos_00.h5'

# if smooth flat
use_flat_smooth = False
# if is_raw is True, set sigma for blurring
smooth_flat_sigma = 10

flat_scale = 1.
norm_scale = 0.5

# smooth normalized xanes images
use_smooth = False
# if use_smooth is True, set sigma for blurring
smooth_xanes_sigma = 10

# if using a mask in alignment calculation
use_mask = True

# use pre-aligned dataset
use_refine = False

# use which image to make a mask
mask_id = 50
# threshold in making a mask
mask_thres = 0.35
# use_anchor: if mask_id will be used as an anchor for entire serires alignment
use_anchor = True
# alignment methods: 'sr', 'pc', and 'mpc'; for 'mpc', you have to specify a mask
reg_method = 'mpc' 
# if you just want to align a subset of the data, set img_s and img_e
img_s = 0
img_e = 101
# ref_mode can be 'single', 'average', or 'neighbor'
ref_mode = 'neighbor'
# overlap_ratio is a parameter controlling 'mpc' method behavior
overlap_ratio = 0.3

print('\033[04m\033[01m\033[34m     Section 2 finished     ')   

     Section 2 finished     


Section 3: read raw data

In [3]:
roi = [roi[0], roi[0]+roi[1], roi[2], roi[2]+roi[3]]
if multipos:
    fn = Path(fn_template.format(scan_id, str(pos).zfill(2)))
else:
    fn = Path(fn_template.format(scan_id))

f = h5py.File(fn, 'r')
if use_refine:
    print('use refine')
    imgs = f['/registration/registered_image'][img_s:img_e]
    eng = f['/raw_data_info/X_eng'][img_s:img_e]
elif is_raw:
    print('is raw')
    imgs = f['/img_xanes'][img_s:img_e, roi[0]:roi[1], roi[2]:roi[3]]
    if use_alternative_flat:
        f1 = h5py.File(flat_name, 'r')
        flat = f1['/img_bkg'][img_s:img_e, roi[0]:roi[1], roi[2]:roi[3]]*flat_scale
        
    else:
        flat = f['/img_bkg'][img_s:img_e, roi[0]:roi[1], roi[2]:roi[3]]*flat_scale
    dark = f['/img_dark'][:, roi[0]:roi[1], roi[2]:roi[3]]
    eng = f['/X_eng'][img_s:img_e]
else:   
    print('is not raw')
    imgs = f['/img_xanes'][img_s:img_e, roi[0]:roi[1], roi[2]:roi[3]]
    eng = f['/X_eng'][img_s:img_e]
f.close()

if (not use_refine) and (is_raw):
    if use_flat_smooth:
        xiu.smooth_flat(flat, smooth_flat_sigma)
        xiu.normalize_raw(imgs, flat, dark)
else:
    imgs *= norm_scale

if not use_refine:
    imgs[:] = -np.log(imgs)[:]
imgs[np.isinf(imgs)] = 0
imgs[np.isnan(imgs)] = 0

if use_smooth:    
    original_imgs = deepcopy(imgs)
#    if not use_refine:
#        original_imgs[:] = -np.log(original_imgs)[:]
#        original_imgs[np.isinf(original_imgs)] = 0
#        original_imgs[np.isnan(original_imgs)] = 0
    imgs[:] = gaussian_filter(imgs, smooth_xanes_sigma)[:]
    
print('\033[04m\033[01m\033[34m     Section 3 finished     ')         

is not raw
     Section 3 finished     


/home/xiao/software/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in log


Section : make mask

In [4]:
try:
    for ip in imgp:
        ip.close()
    imgp = []
except:
    imgp = []

if use_mask is True:
    mask = np.ndarray([imgs.shape[1], imgs.shape[2]])
    mask[:] = skm.binary_dilation(spn.gaussian_filter(imgs[50],10) > mask_thres, np.ones([10,10]))
    mask = np.tile(mask, [img_e - img_s, 1, 1])

    ijui.show(ij.py.to_java(imgs[int(imgs.shape[0]/2)]))
    imp1 = WindowManager.getCurrentImage()
    imp1.setTitle('imgs')
    imgp.append(imp1)
    
    ijui.show(ij.py.to_java(imgs[int(imgs.shape[0]/2)]*mask[0]))
    imp1 = WindowManager.getCurrentImage()
    imp1.setTitle('imgs*mask')
    imgp.append(imp1)
else:
    mask = np.ones([img_e-img_s, imgs.shape[1], imgs.shape[2]])
    
print('\033[04m\033[01m\033[34m     Section 4 finished     ')   

     Section 4 finished     


Section 5: registration

In [5]:
reg = xr.regtools(method=reg_method, mode='TRANSLATION')
reg.set_mask(mask) 
if use_anchor:
    reg.cal_set_anchor(img_s, img_e, mask_id)
reg.set_saving(save_path=fn.parents[0], fn='aligned_' + fn.name)
reg.set_moving_data(imgs)
reg.set_raw_data_info(**{'X_eng': eng[img_s:img_e], 'roi': roi, 'mask': mask})
reg.reg_xanes2D_chunk(overlap_ratio=overlap_ratio, method=reg_method, ref_mode=ref_mode, save=True) 
if use_smooth:
    reg.translate_2D_stack(original_imgs)
    #reg.set_moving_data(original_imgs)
    reg.save_reg_result(dtype='2D_XANES', data=original_imgs)
reg.fit_metrics('si', 'mse', 'nrmse', order=3, plot=True) 

print('\033[04m\033[01m\033[34m     Section 5 finished     ')   

The registration results will be saved in /NSLS2/xf18id1/users/2020Q1/MATEUSZ_Proposal_305729/aligned_xanes_scan2_id_53210.h5


TypeError: One of data, shape or dtype must be specified

In [22]:
print(reg.sec_shift.shape)
print(reg.shift.shape)
print(original_imgs.shape)
print(reg.shift[0])
print(reg.shift[1])
print(np.matmul(reg.shift[0], reg.shift[1]))
print(reg.savefn)
#f.close()
#f = h5py.File(reg.savefn, 'a')

AttributeError: 'regtools' object has no attribute 'sec_shift'

In [ ]:
#print(reg.shift)
print(reg.moving.dtype)
reg.save_reg_result(dtype='2D_XANES')

In [ ]:
print(fn)
out = '/NSLS2/xf18id1/users/2019Q3/XU_ZHENGRUI_Proposal_305094/test_multipos_2D_xanes_scan2_id_26392_repeat_00_pos_00.h5'
f = h5py.File(out, 'a')
g = f.create_group('registration')
g.create_dataset('registered_image', data=reg.moving.astype(np.float32))
f.close()

Section 5: check xanes spectrum at a given position or in a region

In [ ]:
#plt.plot(reg.moving[:, int(reg.moving.shape[1]/2):int(reg.moving.shape[1]/2+3), 
#                    int(reg.moving.shape[2]/2):int(reg.moving.shape[2]/2+3)].mean(axis=(1,2)))
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(eng, reg.moving[:, 500:503, 500:503].mean(axis=(1,2)))
plt.show()

In [ ]:
fn_template = '/NSLS2/xf18id1/users/2019Q3/XU_ZHENGRUI_Proposal_305094/aligned_multipos_2D_xanes_scan2_id_{0}_repeat_00_pos_00.h5'
# scan number
scan_id = 26340
fn = Path(fn_template.format(scan_id))

f = h5py.File(fn, 'r')
img2 = gaussian_filter(f['/registration/registered_image'][80], 5)
img1 = gaussian_filter(f['/registration/registered_image'][96:101].reshape([-1, img2.shape[0], img2.shape[1]]).mean(axis=0), 5)
f.close()

#img1 = gaussian_filter(img1.mean(axis=0), 10)
#img2[:] = gaussian_filter(img2, 10)[:]
print(img1.shape, img2.shape, mask.shape)

# method: 'pc', 'mpc', 'sr'
reg2 = xr.regtools(method='sr', mode='TRANSLATION')
reg2.set_mask(mask[0])     
reg2.set_saving(save_path=fn.parents[0], fn='aligned_' + fn.name)
reg2.set_fixed_data(img2)
reg2.set_moving_data(img1)
reg2.set_raw_data_info(**{'X_eng': eng[img_s:img_e], 'roi': roi, 'mask': mask[0]})
reg2.reg_2D_to_ref(dtype='3D_XANES', save=False)

fig = plt.figure()
ax1 = fig.add_subplot(121)
ax1.imshow(img2 - img1, vmin=-0.5, vmax=0.5)
ax1.set_title('img2 - img1')
ax2 = fig.add_subplot(122)
ax2.imshow(reg2.moving - reg2.fixed, vmin=-0.5, vmax=0.5)
ax2.set_title('reg2.moving - reg2.fixed')
plt.show()

In [ ]:
f = h5py.File(fn, 'r')
img = f['/registration/registered_image'][:]
f.close()

# used when method = 'pc' or 'mpc'
#reg2.shift = np.concatenate((np.tile(np.array([0, 0]),[90, 1]), np.tile(np.array([-0.87, -8.69]),[61, 1])), axis=0)

# used when method = 'sr'
print(reg2.shift.shape)
#reg2.shift = np.concatenate((np.tile(np.identity(3), [90, 1, 1]), np.tile(reg2.shift, [5, 1, 1]), np.tile(np.identity(3), [6, 1, 1])), axis=0)
reg2.shift = np.concatenate((np.tile(np.identity(3), [94, 1, 1]), np.tile(reg2.shift, [4, 1, 1]), np.tile(np.identity(3), [0, 1, 1])), axis=0)


reg2.set_moving_data(img)
reg2.translate_xanes2D_stack(reg2.moving)
reg2.save_reg_result(dtype='2D_XANES')

In [ ]:
print(np.tile(np.identity(3), [95, 1, 1]).shape, np.tile(reg2.shift, [5, 1, 1]).shape, np.tile(np.identity(3), [0, 1, 1]).shape)

In [ ]:
print(np.identity(3))

In [8]:
help([].index)

Help on built-in function index:

index(...) method of builtins.list instance
    L.index(value, [start, [stop]]) -> integer -- return first index of value.
    Raises ValueError if the value is not present.

